# Example: Compare different methods for a single plume

In this example, we compare how the different methods are applied to a single emission plume. The example uses synthetic CO2 and NO2 observations of the Jänschwalde power plant plume from the SMARTCARB dataset. The focus is generating figures demonstrating how the methods are applied to the plume.

In [ ]:
import os

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import shapely.geometry
import xarray as xr

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches': 'tight'}
plt.rcParams['figure.dpi'] = 100

import ddeq
import ucat

# A small domain around Jänschwalde power plant
DOMAIN = ddeq.misc.Domain('JAE', 14, 51.4, 16, 52.2)
CRS = ccrs.UTM(32)

# Path to test dataset incl in ddeq
from ddeq import DATA_PATH

# Load list of point source locations
sources = ddeq.misc.read_point_sources().sel(source=['Janschwalde'])

# Rough priors for source strength (in kg/s) and decay time (in seconds) used by some methods.
PRIORS = {
    'Janschwalde': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60**2},
    }
}

Read and visualize SMARTCARB example file

In [ ]:
filename = os.path.join(DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')
data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='medium',
                                            no2_noise_scenario='high')

In [ ]:
fig = ddeq.vis.show_level2(data, 'CO2', gas='CO2', sources=sources,
                         domain=DOMAIN, draw_gridlines=True)

# use bbox_inches='tight' to include lon/lat labels in saved image
plt.savefig('test.png', bbox_inches='tight')

Read wind speed at sources used by CSF, IME and GP and 2D wind field used by LCSF

In [ ]:
time = pd.Timestamp(data.time.to_pandas())
winds = ddeq.wind.read_smartcarb(time, sources.lon, sources.lat, data_path=DATA_PATH)

wind_field = ddeq.wind.read_field(os.path.join(DATA_PATH, 'SMARTCARB_winds_2015042311.nc'), product='SMARTCARB')

Plume detection and pre-processing (for CSF, IME and GP method)

In [ ]:
data = ddeq.dplume.detect_plumes(data, sources, variable='NO2', variable_std='NO2_std',
                                 filter_type='gaussian', filter_size=0.5, crs=CRS)

In [ ]:
data = ddeq.curves.fit_to_detections(data, n_nodes=3, force_origin=True, use_weights=True)
data = ddeq.curves.compute_natural_coords(data)
data = ddeq.curves.compute_plume_areas(data)

In [ ]:
for gas in ["CO2", "NO2"]:

    # estimate background
    data = ddeq.background.estimate(data, gas)

    # compute CO2/NO2 enhancement
    data = ddeq.emissions.compute_plume_signal(data, gas)

    # convert ppm to kg/m2
    for variable in [
        gas,
        f"{gas}_estimated_background",
        f"{gas}_minus_estimated_background",
    ]:
        ddeq.emissions.convert_units(data, gas, variable)

## LCSF
Run LCSF method and visualize plume.

In [ ]:
lcsf_results = ddeq.lcsf.estimate_emissions(data, wind_field, sources, ['CO2'], priors=PRIORS, all_diags=True)

In [ ]:
fig = ddeq.vis.visualize_lcsf_plume(data, lcsf_results, 'CO2', sources, source_name='Janschwalde',
                                    domain=DOMAIN, do_zoom=False, line_color='b')
plt.savefig('lcsf-example.png', bbox_inches='tight')
plt.savefig('lcsf-example.pdf', bbox_inches='tight')

## Gauss
Run GP method and visualize the result

In [ ]:
data, gp_results = ddeq.gauss.estimate_emissions(data, winds, sources, ['NO2', 'CO2'], priors=PRIORS)

In [ ]:
# convert from kg m-2 to ppmv
gauss_fit = ucat.convert_columns(data['CO2_plume_model_mass'] + gp_results['CO2_background'], 'kg m-2', 'ppmv', molar_mass='CO2')[:,:,0]
gauss_fit += data.CO2_estimated_background

In [ ]:
fig = ddeq.vis.show_level2(
    data, data['CO2'], gas='CO2', domain=DOMAIN, winds=winds,
    do_zoom=False, crs=CRS, zoom_on="Janschwalde"
)

s = slice(450,550), slice(40, None)
fig.axes[0].contour(data.lon[s], data.lat[s], gauss_fit[s], transform=ccrs.PlateCarree(), colors='b', levels=np.arange(403, 407, 0.5))

plt.savefig('gp-example.png', bbox_inches='tight')
plt.savefig('gp-example.pdf', bbox_inches='tight')

## CSF
Visualize the CSF method.

In [ ]:
fig = ddeq.vis.show_level2(
    data, data['CO2'], gas='CO2', domain=DOMAIN, winds=winds,
    do_zoom=False, crs=CRS,
)
plt.savefig('csf-example.png', bbox_inches='tight')
plt.savefig('csf-example.pdf', bbox_inches='tight')

## IME
Visualize IME method.

In [ ]:
fig = ddeq.vis.show_level2(
    data, data['CO2'], gas='CO2', domain=DOMAIN, winds=winds,
    do_zoom=False, crs=CRS, draw_plume_area=False,
)

# calculate integration area and convert to convex hull line for visualization
this = data.sel(source='Janschwalde')
area = this['plume_area']
L = np.max(this['xp'].values[area]) - 10e3
area = (0 <= this['xp']) & (this['xp'] <= L) & area
lon, lat = this.lon.values[area], this.lat.values[area]
pts = shapely.geometry.MultiPoint(np.transpose([lon.flatten(), lat.flatten()]))
x,y = np.transpose(pts.convex_hull.exterior.coords[:])

fig.axes[0].plot(x.flatten(), y.flatten(), 'b-', ms=1, transform=ccrs.PlateCarree())
plt.savefig('ime-example.png', bbox_inches='tight')
plt.savefig('ime-example.pdf', bbox_inches='tight')